In [1]:
import time

from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard

from model import *
from data import *
import ImageTB

Using TensorFlow backend.


In [2]:
MODEL_NAME = 'Unet'
# MODEL_NAME = 'RefineNet'
# MODEL_NAME = 'Segnet'
TRAIN_PATH = 'data/sky/iphone500'
VAL_PATH = 'data/sky/iphone500'

LOGS_PATH = './../../../log'
if not os.path.exists(LOGS_PATH):
    os.makedirs(LOGS_PATH)
    
WEIGHTS_PATH = './models'
if not os.path.exists(WEIGHTS_PATH):
    os.makedirs(WEIGHTS_PATH)
    
BATCH_SIZE = 4
VAL_BATCH_SIZE = 5

## Lr reduce

In [3]:
lr_reducer = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

## Checkpoint

In [4]:
weights_name = MODEL_NAME + '.{epoch:02d}-{val_acc:.2f}.hdf5'
weights_dir = os.path.join(WEIGHTS_PATH, weights_name)
checkpoint = ModelCheckpoint(weights_dir, monitor='acc', verbose=1, save_best_only=True, mode='max')

## Model

In [5]:
model = unet()
# model = RefineNet()
# model = segnet()
# model.summary()

/home/dmitrii/.virtenvs/FaceIidentification/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/home/dmitrii/sky_detect/Semantic-Segmentation-Suite/UNET/sky-segmentation/keras_unet/model.py:79: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


## Generators

In [6]:
data_gen_args = dict(
    rotation_range=0.3,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [7]:
train_generator = BatchGenerator(
    batch_size=BATCH_SIZE,
    train_path=TRAIN_PATH,
    image_folder='train',
    mask_folder='train_labels',
    aug_dict=data_gen_args,
    save_to_dir = None)

val_generator = BatchGenerator(
    batch_size=VAL_BATCH_SIZE,
    train_path=VAL_PATH,
    image_folder='val',
    mask_folder='val_labels',
    aug_dict=data_gen_args,
    save_to_dir = None)

Found 1263 images belonging to 1 classes.
Found 1263 images belonging to 1 classes.
Found 26 images belonging to 1 classes.
Found 26 images belonging to 1 classes.


## TensorBoard

In [ ]:
CURRENT_TIME = str(time.strftime("%Y%m%d-%H%M%S"))    
log_name = "{}_{}".format(MODEL_NAME, CURRENT_TIME)
log_dir = os.path.join(LOGS_PATH, log_name)
os.makedirs(log_dir)
tbCallBack = ImageTB.ImagesTensorBoard(log_dir=log_dir, generator=val_generator)

## Early stopping

In [ ]:
erlStopping = ImageTB.EarlyStoppingByLossVal(verbose=1)

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=500,
    callbacks=[
            lr_reducer,
            tbCallBack,
            checkpoint,
#             erlStopping
        ],
    validation_data=val_generator,
    validation_steps=len(val_generator),
    
    max_queue_size=8,
    workers=4,
    use_multiprocessing=True)

Epoch 1/500
314/315 [============================>.] - ETA: 0s - loss: 0.1729 - acc: 0.9418Epoch 1/500
Epoch 1/500
315/315 [==============================] - 159s 505ms/step - loss: 0.1728 - acc: 0.9419 - val_loss: 0.1741 - val_acc: 0.9601

Epoch 00001: acc improved from -inf to 0.94195, saving model to ./models/Unet.01-0.96.hdf5
Epoch 2/500
315/315 [==============================] - 132s 420ms/step - loss: 0.1143 - acc: 0.9660 - val_loss: 0.1318 - val_acc: 0.9517

Epoch 00002: acc improved from 0.94195 to 0.96596, saving model to ./models/Unet.02-0.95.hdf5
Epoch 3/500
315/315 [==============================] - 126s 399ms/step - loss: 0.0873 - acc: 0.9729 - val_loss: 0.0344 - val_acc: 0.9875

Epoch 00003: acc improved from 0.96596 to 0.97287, saving model to ./models/Unet.03-0.99.hdf5
Epoch 4/500
315/315 [==============================] - 125s 397ms/step - loss: 0.0769 - acc: 0.9762 - val_loss: 0.0421 - val_acc: 0.9849

Epoch 00004: acc improved from 0.97287 to 0.97623, saving model to